In [1]:
import numpy as np

In [26]:
opcodeInput=np.loadtxt("input_day9.txt",delimiter=",",dtype=np.int64)

In [21]:
opcodeInput=np.array([109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99])

In [80]:
opcodeInput=np.array([1102,34915192,34915192,7,4,7,99,0],dtype=np.int64)
# python int would be without size and thus upcast if overflown, but numpy type is fixed 

In [60]:
opcodeInput=np.array([104,1125899906842624,99])

In [3]:
class OpCodeManager:
    def __init__(self, opcode):
        self.opcode = opcode
        self.relativeBaseIdx=0
    def __getitem__(self, idx):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        return self.opcode[idx]
    def __setitem__(self, idx, value):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        self.opcode[idx] = value

In [32]:
def decodeCommand(command):
    code=command%100
    p1Mode=int(command/100)%10
    p2Mode=int(command/1000)%10
    p3Mode=int(command/10000)%10
    return [code, p1Mode, p2Mode, p3Mode]

def getArgument(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return opcode[parameter]
    elif mode == 1:
        # immediate mode
        return parameter
    else:
        # relative mode
        return opcode[opcode.relativeBaseIdx+parameter]
    
def getWritePosition(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return parameter
    elif mode == 1:
        # immediate mode
        print("ERROR! Immediate mode not intended for writing")
        print(mode, parameter, opcode)
    else:
        # relative mode
        return opcode.relativeBaseIdx+parameter

In [43]:
def processOpcode(opcode, idx):
    
    while True:
       
        decoded = decodeCommand(opcode[idx])

        if decoded[0] == 99:
            return
        elif decoded[0] == 1:
            #add p1 p2 p3
            p1 = opcode[idx+1]
            p2 = opcode[idx+2]
            p3 = opcode[idx+3]
            arg1 = getArgument(opcode,decoded[1],p1)
            arg2 = getArgument(opcode,decoded[2],p2)
            arg3 = getWritePosition(opcode,decoded[3],p3)
            opcode[arg3] = arg1 + arg2
            idx+=4
        elif decoded[0] == 2:
            #multiply p1 p2 p3
            p1 = opcode[idx+1]
            p2 = opcode[idx+2]
            p3 = opcode[idx+3]
            arg1 = getArgument(opcode,decoded[1],p1)
            arg2 = getArgument(opcode,decoded[2],p2)
            arg3 = getWritePosition(opcode,decoded[3],p3)
            opcode[arg3] = arg1 * arg2
            idx+=4
        elif decoded[0] == 3:
            #input p1
            p1 = opcode[idx+1]
            arg1 = getWritePosition(opcode,decoded[1],p1)
            var = input("Please enter input: ")
            opcode[arg1] = np.int64(var)
            idx+=2
        elif decoded[0] == 4:
            #output p1
            p1 = opcode[idx+1]
            arg1 = getArgument(opcode,decoded[1],p1)
            print("Output: ", arg1)
            idx+=2
        elif decoded[0] == 5:
            #jump-if-true p1 p2
            p1 = opcode[idx+1]
            p2 = opcode[idx+2]
            arg1 = getArgument(opcode,decoded[1],p1)
            arg2 = getArgument(opcode,decoded[2],p2)
            if arg1 != 0:
                idx=arg2
            else:      
                idx=idx+3
        elif decoded[0] == 6:
            #jump-if-false p1 p2
            p1 = opcode[idx+1]
            p2 = opcode[idx+2]
            arg1 = getArgument(opcode,decoded[1],p1)
            arg2 = getArgument(opcode,decoded[2],p2)
            if arg1 == 0:
                idx=arg2
            else:      
                idx+=3
        elif decoded[0] == 7:
            #less-than p1 p2 p3
            p1 = opcode[idx+1]
            p2 = opcode[idx+2]
            p3 = opcode[idx+3]
            arg1 = getArgument(opcode,decoded[1],p1)
            arg2 = getArgument(opcode,decoded[2],p2)
            arg3 = getWritePosition(opcode,decoded[3],p3)
            if arg1 < arg2:
                opcode[arg3]=1
            else:      
                opcode[arg3]=0
            idx+=4
        elif decoded[0] == 8:
            #equals p1 p2 p3
            p1 = opcode[idx+1]
            p2 = opcode[idx+2]
            p3 = opcode[idx+3]
            arg1 = getArgument(opcode,decoded[1],p1)
            arg2 = getArgument(opcode,decoded[2],p2)
            arg3 = getWritePosition(opcode,decoded[3],p3)
            if arg1 == arg2:
                opcode[arg3]=1
            else:      
                opcode[arg3]=0
            idx+=4
        elif decoded[0] == 9:
            # change relative base p1
            p1 = opcode[idx+1]
            arg1 = getArgument(opcode,decoded[1],p1)
            opcode.relativeBaseIdx += arg1
            idx+=2
        else:
            print("Invalid code ", opcode, idx, opcode[idx], decoded)
            return

In [44]:
#task 1
opcodeMgr=OpCodeManager(opcodeInput.copy())
processOpcode(opcodeMgr,0)

Please enter input: 1
Output:  3780860499


In [45]:
#task 2
opcodeMgr=OpCodeManager(opcodeInput.copy())
processOpcode(opcodeMgr,0)

Please enter input: 2
Output:  33343


int